# TraceLog.SQL

## Domain

In [ ]:
type SelectCommand = seq<Conditioner>
and FromCommand    = string
and WhereCommand   = seq<Conditioner> 
and Query = 
    { Select : SelectCommand 
      From   : FromCommand 
      Where  : WhereCommand }
and Condition = 
    {  Conditioner      : Conditioner
       ConditionType    : ConditionType
       ConditionalValue : ConditionalValue }
and Conditioner = 
    | EventProperty of ConditionerEventProperty
    | ReservedWord  of ReservedWord 
and ConditionerEventProperty = 
    { ConditionerEvent    : ConditionerEvent 
      ConditionerProperty : ConditionerProperty }
and ConditionerEvent    = string
and ConditionerProperty = string
and ConditionType = 
    | LessThan
    | LessThanEqualTo
    | GreaterThan
    | GreaterThanEqualTo
    | Equal
    | NotEqual
and ConditionalValue =
    | DoubleValue of double
    | StringValue of string
and ReservedWord = 
    | ProcessName   of string
    | ProcessID     of int
    | ThreadID      of int
    | ProcessNumber of int
    | ID            of string
    | ClrID         of string

let ReservedWords : Set<string> = 
    [ "ProcessName"; "ProcessID"; "ThreadID"; "HasStack"; "ProcessNumber"; "ID"; "ClrID" ]
    |> Set.ofList

Example of Query:

```
    SELECT GC/AllocationTick.AllocationAmount, GC/HeapStats.Generation0, ProcessName, ThreadId FROM <FilePath> 
   WHERE ProcessName = devenv and 
         ProcessId   = 10     and 
         GC/AllocationTick.AllocationAmount >= 5000
    ORDER BY Timestamp 
```

Or more generally:

```
   SELECT ColumnNames FROM TracePath
   WHERE Condition1 and 
         Condition2 and
         Condition3 ... 
    ORDER BY OrderingList // By default, Timestamp.
```

Reserved Columns:

1. ProcessName
2. ProcessID
3. ThreadID
4. HasStack
5. ProcessNumber
6. ID
7. ClrID
8. TimeStamp
9. TimeStampRelativeMSec

In [ ]:
#r "nuget:Microsoft.Diagnostics.Tracing.TraceEvent"

open Microsoft.Diagnostics.Tracing;
open Microsoft.Diagnostics.Tracing.Etlx;
open Microsoft.Diagnostics.Tracing.Session;
open Microsoft.Diagnostics.Tracing.Parsers.Clr;

let ETL_FILEPATH = @"C:\Users\mukun\OneDrive\Documents\CallstackShmuff.etl\CallstackShmuff.etl"

let traceLog = TraceLog.OpenOrConvert ETL_FILEPATH
traceLog.Events
|> Seq.take 5

Installed Packages Microsoft.Diagnostics.Tracing.TraceEvent, 2.0.74

index type BufferSize Version ProviderVersion NumberOfProcessors EndTime TimerResolution MaxFileSize LogFileMode BuffersWritten StartBuffers PointerSize EventsLost CPUSpeed UTCOffsetMinutes BootTime PerfFreq StartTime ReservedFlags BuffersLost SessionName .. 0 Microsoft.Diagnostics.Tracing.Parsers.Kernel.EventTraceHeaderTraceData 65536 131082 22000 8 2021-12-04 08:15:21Z 156250 800 67174401 5592 1 8 0 2803 480 2021-12-04 07:47:01Z 10000000 2021-12-04 08:14:46Z 1 0 Relogger 1 Microsoft.Diagnostics.Tracing.Parsers.DynamicTraceEventData 2 8 2 Microsoft.Diagnostics.Tracing.Parsers.Kernel.HeaderExtensionTraceData 2 8 3 Microsoft.Diagnostics.Tracing.Parsers.DynamicTraceEventData 2 8 4 Microsoft.Diagnostics.Tracing.Parsers.Kernel.SystemPathsTraceData 0 8

## Parsing Conditions

### Splitting Multiple Conditions

In [ ]:
let conditions = "GC/AllocationTick.AllocationAmount > 20000 and ProcessName = \"devenv\""
let splitConditions : string[] = conditions.Split("and", StringSplitOptions.RemoveEmptyEntries)
splitConditions

index,value
0,GC/AllocationTick.AllocationAmount > 20000
1,"ProcessName = ""devenv"""


#### Parsing The Conditioner

2 types:

- GC/AllocationTick.AllocationAmount > 20000
- ProcessName = "devenv"

### Split Full Condition 

In [ ]:
let condition1 = "GC/AllocationTick.AllocationAmount > 20000"
let s = condition1.Split(" ", StringSplitOptions.RemoveEmptyEntries)
let f  = s.[0]
let cc = s.[1]
let v  = s.[2]

// precondition: check if there are 3 
let condition2 = "ProcessName = \"devenv\""
let condition2WithoutQuotes = condition2.Replace("\"", "")
let s2  = condition2.Split(" ", StringSplitOptions.RemoveEmptyEntries)
let f2  = s2.[0]
let cc2 = s2.[1]
let v2  = s2.[2]

s2

index,value
0,ProcessName
1,=
2,"""devenv"""


### Split For Conditioner

#### Reserved Keyword

In [ ]:
let parseReservedKeyword (secondSplit : string) : Conditioner =
    match (ReservedWords.Contains secondSplit) with
    | true  -> 
        match secondSplit with 
            | "ProcessName"   -> Conditioner.ReservedWord( ProcessName f2 )
            | "ProcessID"     -> Conditioner.ReservedWord( ProcessID (int (f2)) )
            | "ThreadID"      -> Conditioner.ReservedWord( ThreadID (int (f2)) )
            | "ProcessNumber" -> Conditioner.ReservedWord( ProcessNumber (int (f2)))
            | "ID"            -> Conditioner.ReservedWord( ID f2 )
            | "ClrID"         -> Conditioner.ReservedWord( ClrID f2 )
    | false ->
        invalidArg secondSplit "Non Reserved Word given"

display(parseReservedKeyword f2)
display(parseReservedKeyword "GC/AllocationTick.AllocationAmount")

Item
"{ ProcessName ""ProcessName"": Item: ProcessName }"


Error: System.ArgumentException: Non Reserved Word given (Parameter 'GC/AllocationTick.AllocationAmount')
   at FSI_0014.parseReservedKeyword(String secondSplit)
   at <StartupCode$FSI_0014>.$FSI_0014.main@()

#### Event Property and Keyword

In [ ]:
let parseConditionerEventProperty(secondSplit : string) : Conditioner =
    match (ReservedWords.Contains secondSplit) with
    | false -> 
        let splitEventAndProperty = secondSplit.Split(".", StringSplitOptions.RemoveEmptyEntries) 
        if splitEventAndProperty.Length < 2 then
            invalidArg secondSplit "The argument is neither a reserved keyword nor a properly formatted trace event and property; the format if the latter should be: EventName.Property"
        else
            Conditioner.EventProperty{ ConditionerEvent =  splitEventAndProperty.[0]; ConditionerProperty = splitEventAndProperty.[1] }
    | true ->
        invalidArg secondSplit "Reserved Word given"

display(parseConditionerEventProperty "GC/AllocationTick.AllocationAmount")
display(parseConditionerEventProperty f2)

Item
"{ { ConditionerEvent = ""GC/AllocationTick"" ConditionerProperty = ""AllocationAmount"" }: ConditionerEvent: GC/AllocationTick, ConditionerProperty: AllocationAmount }"


Error: System.ArgumentException: Reserved Word given (Parameter 'ProcessName')
   at FSI_0018.parseConditionerEventProperty(String secondSplit)
   at <StartupCode$FSI_0018>.$FSI_0018.main@()

#### Combining Reserved Keyword and Event Property 

In [ ]:
let parseConditioner (conditionerAsString: string) : Conditioner = 
    match (ReservedWords.Contains conditionerAsString) with
    | true ->
        match conditionerAsString with 
            | "ProcessName"   -> Conditioner.ReservedWord( ProcessName f2 )
            | "ProcessID"     -> Conditioner.ReservedWord( ProcessID (int (f2)) )
            | "ThreadID"      -> Conditioner.ReservedWord( ThreadID (int (f2)) )
            | "ProcessNumber" -> Conditioner.ReservedWord( ProcessNumber (int (f2)))
            | "ID"            -> Conditioner.ReservedWord( ID f2 )
            | "ClrID"         -> Conditioner.ReservedWord( ClrID f2 )
    | false -> 
        let splitEventAndProperty = conditionerAsString.Split(".", StringSplitOptions.RemoveEmptyEntries) 
        if splitEventAndProperty.Length < 2 then
            invalidArg conditionerAsString "The argument is neither a reserved keyword nor a properly formatted trace event and property; the format if the latter should be: EventName.Property"
        else
            Conditioner.EventProperty{ ConditionerEvent =  splitEventAndProperty.[0]; ConditionerProperty = splitEventAndProperty.[1] }

In [ ]:
let condition1   = "GC/AllocationTick.AllocationAmount > 20000"
let conditioner1 = condition1.Split(" ", StringSplitOptions.RemoveEmptyEntries)[0]
display(parseConditioner conditioner1)

let condition2   = "ProcessName = \"devenv\""
let conditioner2 = condition2.Split(" ", StringSplitOptions.RemoveEmptyEntries)[0]
display(parseConditioner conditioner2)

Item
"{ { ConditionerEvent = ""GC/AllocationTick"" ConditionerProperty = ""AllocationAmount"" }: ConditionerEvent: GC/AllocationTick, ConditionerProperty: AllocationAmount }"


Item
"{ ProcessName ""ProcessName"": Item: ProcessName }"


### Condition Parser

In [ ]:
let parseCondition (conditionAsString : string) : Condition =  

    let splitCondition = conditionAsString.Split(" ", StringSplitOptions.RemoveEmptyEntries) 

    // Conditioner
    let parseConditioner : Conditioner = 
        let conditionerAsString = splitCondition.[0] 
        match (ReservedWords.Contains conditionerAsString) with
        | true ->
            match conditionerAsString with 
                | "ProcessName"   -> Conditioner.ReservedWord( ProcessName f2 )
                | "ProcessID"     -> Conditioner.ReservedWord( ProcessID (int (f2)) )
                | "ThreadID"      -> Conditioner.ReservedWord( ThreadID (int (f2)) )
                | "ProcessNumber" -> Conditioner.ReservedWord( ProcessNumber (int (f2)))
                | "ID"            -> Conditioner.ReservedWord( ID f2 )
                | "ClrID"         -> Conditioner.ReservedWord( ClrID f2 )
        | false -> 
            let splitEventAndProperty = conditionerAsString.Split(".", StringSplitOptions.RemoveEmptyEntries) 
            if splitEventAndProperty.Length < 2 then
                invalidArg conditionerAsString "The argument is neither a reserved keyword nor a properly formatted trace event and property; the format if the latter should be: EventName.Property"
            else
                Conditioner.EventProperty{ ConditionerEvent =  splitEventAndProperty.[0]; ConditionerProperty = splitEventAndProperty.[1] }

    // Condition Type
    let parseConditionType : ConditionType =
        match splitCondition.[1].ToLower() with
        | ">"  | "greaterthan"                                 -> ConditionType.GreaterThan 
        | "<"  | "lessthan"                                    -> ConditionType.LessThan
        | ">=" | "greaterthanequalto" | "greaterthanorequalto" -> ConditionType.GreaterThanEqualTo
        | "<=" | "lessthanequalto"    | "lessthanorequalto"    -> ConditionType.LessThanEqualTo
        | "="  | "equal"              | "equals"               -> ConditionType.Equal
        | "!=" | "notequal"                                    -> ConditionType.NotEqual
        | _                                                    -> invalidArg (nameof splitCondition) ("${splitCondition.[1]} is an unrecognized condition type.")

    // Condition Value
    let parseConditionValue : ConditionalValue =
        let conditionalValueAsString = splitCondition.[2].ToLower()
        let checkDouble, doubleValue = Double.TryParse conditionalValueAsString 
        match checkDouble, doubleValue with
        | true, v  -> ConditionalValue.DoubleValue(v)
        | false, v -> ConditionalValue.StringValue(conditionalValueAsString.Replace("'", ""))

    { Conditioner = parseConditioner; ConditionType = parseConditionType; ConditionalValue = parseConditionValue }
        
let parseConditions (conditionAsString : string) : seq<Condition> = 
    let splitCondition : string[] = conditionAsString.Split("AND", StringSplitOptions.RemoveEmptyEntries)

    // TODO: Add NOT / OR Logic.
    (*
    let rec matchConditions (runningConditions : seq<Condition>) (remaining : string[]) =
        match conditionAsString with
        | x     :: xs ->  
        | x :: "AND" :: xs -> 
            seq
        | x :: "OR" :: xs  ->
        | []          -> runningConditions
    *)

    splitCondition
    |> Seq.map(parseCondition)

### Tests

#### Parse Condition

In [ ]:
let parseCondition1 = "GC/AllocationTick.AllocationAmount > 20000"
display(parseCondition parseCondition1)

let parseCondition2 = "ProcessName = 'devenv'"
display(parseCondition parseCondition2)

#### Parse Conditions

In [ ]:
let parseConditions1 = "GC/AllocationTick.AllocationAmount > 20000 AND ProcessName = 'devenv'"
display(parseConditions parseConditions1)

index Conditioner ConditionType ConditionalValue 0 Item { { ConditionerEvent = "GC/AllocationTick"
 ConditionerProperty = "AllocationAmount" }: ConditionerEvent: GC/AllocationTick, ConditionerProperty: AllocationAmount } GreaterThan Item 20000 1 Item { ProcessName "ProcessName": Item: ProcessName } Equal Item devenv

## Parsing Trace Log Location

In [ ]:
let parseTraceLocation (query : string) : FromCommand = 
    let splitTraceLog = query.Split(" ", StringSplitOptions.RemoveEmptyEntries ||| StringSplitOptions.TrimEntries)
    let idxOfFrom : int = 
        splitTraceLog
        |> Array.findIndex(fun s -> ( s.ToLower() = "from" ))

    let error = "Specify the the trace location using FROM i.e. Select * FROM <TracePath>"
    // FROM not found => error.
    if idxOfFrom = -1 then invalidArg query error
    // TraceLogPath not in the query => error.
    elif idxOfFrom + 1 = splitTraceLog.Length then invalidArg query error
    else
        splitTraceLog.[idxOfFrom + 1]

In [ ]:
let parsingTraceLogLocation = "SELECT GC/AllocationTick.AllocationAmount, ProcessName FROM tracePath"
parseTraceLocation parsingTraceLogLocation

tracePath

## Parsing Select Columns

In [ ]:
let parseSelectColumns (query : string) = 
    let splitTraceLog = query.Split(" ", StringSplitOptions.RemoveEmptyEntries ||| StringSplitOptions.TrimEntries)

    // precondition checks.
    // Check if select is in query.
    if splitTraceLog.Length < 1 then invalidArg query $"Invalid Query without SELECT and FROM commands: {query}"
    elif splitTraceLog.[0].ToLower() <> "select" then invalidArg query $"Invalid Query - SELECT not supplied: {query}"

    let idxOfSelectedColumns = splitTraceLog |> Array.findIndex(fun s -> s.ToLower() = "from")
    printfn "%A" idxOfSelectedColumns
    if ( idxOfSelectedColumns = -1 ) then invalidArg query $"Invalid Query without FROM command to specify trace log file: {query}"
    
    printfn "%A" splitTraceLog
    //let csvSelectColumns = selectColumns.Split(",", StringSplitOptions.RemoveEmptyEntries ||| StringSplitOptions.TrimEntries)
    //printfn "%A" csvSelectColumns 
    ()

In [ ]:
let parseSelectColumns1 = "SELECT GC/AllocationTick.AllocationAmount, ProcessName  = 'devenv' FROM tracePath"

parseSelectColumns parseSelectColumns1

5
[|"SELECT"; "GC/AllocationTick.AllocationAmount,"; "ProcessName"; "=";
  "'devenv'"; "FROM"; "tracePath"|]


In [ ]:
let parseString (query : string) =
    let splitTraceLog = query.Split(" ", StringSplitOptions.RemoveEmptyEntries ||| StringSplitOptions.TrimEntries)
    let rec r (splitTraceLog : list<string>) (acc : list<string>)=
        match splitTraceLog with
        | "SELECT" :: x :: xs | "select" :: x :: xs  ->
            printfn $"HERE -- {x} x {xs}"
            // Accumulate conditioners.
            r (x :: xs) (x :: acc)
        | "FROM" :: rest ->
            // Process conditioners.
            printfn "%A" acc
            let processed = 
                acc
                |> List.map parseConditioner
            printfn "%A" processed
        | x :: xs ->
            printfn "%A - %A" x xs
            ()
    ()

In [ ]:
parseString parseSelectColumns1